In [1]:
import json
import requests
import minsearch

In [2]:
with open('documents.json','rt') as f_in:
    docs = json.load(f_in)

In [6]:
documents = []

for course_dict in docs:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [9]:
index = minsearch.Index(text_fields=["question", "text", "section"],
                        keyword_fields=["course"]
)

index.fit(documents)

In [10]:
query = "I just discovered the course, can i still join?"
index.search(query)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related question',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks as long as the form is still open and accepting submissions.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything to the last minute.",
  'section': 'General course questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'mlops-zoomcamp'},
 {'text': "Yes, even i